<a href="https://www.kaggle.com/code/pogscafe/wonder3d-kaggle?scriptVersionId=159056204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Installation
You can ignore the red text.

In [ ]:
HOME_DIR = '/kaggle/working'
TEMP_DIR = '/kaggle/temp'
!mkdir /kaggle/temp

import os
from os import path

%cd $HOME_DIR
!pwd
if not path.exists(f'{HOME_DIR}/Wonder3D'):
    get_ipython().system('git clone https://github.com/wandaweb/Wonder3D')
os.chdir(f'{HOME_DIR}/Wonder3D')
get_ipython().system('git pull')
!pip install typing_extensions 
if not path.exists(f'{TEMP_DIR}/ckpts'):
    os.chdir(TEMP_DIR)
    get_ipython().system(f'ls')
    get_ipython().system('git clone https://huggingface.co/camenduru/Wonder3D')
    get_ipython().system('mv Wonder3D ckpts')
if not path.exists(f'{HOME_DIR}/Wonder3D/ckpts'):
    get_ipython().system(f'ln -s {TEMP_DIR}/ckpts {HOME_DIR}/Wonder3D/')
if not path.exists(f'{HOME_DIR}/Wonder3D/sam_pt'):
    os.chdir(f'{HOME_DIR}/Wonder3D')
    get_ipython().system('mkdir sam_pt')
    os.chdir(f'{HOME_DIR}/Wonder3D/sam_pt')
    get_ipython().system('wget https://huggingface.co/segments-arnaud/sam_vit_h/resolve/main/sam_vit_h_4b8939.pth')
 
os.chdir(f'{HOME_DIR}/Wonder3D')
!pip install triton
!pip install gradio==3.48.0
!pip install fire
!pip install opencv-python
!pip install rembg
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install streamlit
!pip install numpy==1.22.4 
!pip install -r requirements.txt
!mamba install openssh -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu118
!pip install gdown
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
if not path.exists(f'tinycudann-1.7-cp310-cp310-linux_x86_64.whl'):
    get_ipython().system('wget "https://j2q5.c17.e2-1.dev/download/pogscafe/tinycudann-1.7-cp310-cp310-linux_x86_64.whl"')
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl
!pip install torch torchvision torchaudio xformers --force-reinstall --index-url https://download.pytorch.org/whl/cu118

# 
# Starting the Web UI

Wait for the line "Running on local URL:  http://127.0.0.1:7860 " and click the long link ending with .remote.moe  
Generating each new set of images will overwrite the previous one. When the images are created, press Stop and continue with the next code block to generate the 3D model.

In [ ]:
# Starting with RemoteMoe
%cd /kaggle/working/Wonder3D
!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python /kaggle/working/Wonder3D/gradio_app.py & ssh -R 80:127.0.0.1:7860 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# 
# Create the model with instant-nsr-pl
Create a 3D model from the last set of generated images. This model will be more detailed than NeuS. The output model is located in /kaggle/working/Wonder3D/instant-nsr-pl/exp/mesh-ortho-out  
You can find a copy of the 3D model in /kaggle/working/outputs

In [ ]:
%cd /kaggle/working/Wonder3D/instant-nsr-pl
!bash run.sh /kaggle/working/Wonder3D/outputs out

# copies the latest output files to /kaggle/working/outputs
import os, datetime
path = '/kaggle/working/Wonder3D/instant-nsr-pl/exp/mesh-ortho-out'
folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
last_folder = max(folders)
os.chdir(os.path.join(path, last_folder))
%cd 'save'
output_folder = '/kaggle/working/outputs/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok = True)
foldername = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
new_folder = os.path.join(output_folder, 'out_'+foldername)
print(new_folder)
os.makedirs(new_folder)
get_ipython().system(f'cp *.* {new_folder}')
print(f'Output copied to: {new_folder}')

### Create downloadable links to output files

In [ ]:
# Create downloadable links to output files
import os
from IPython.display import FileLink
from IPython.display import HTML
%cd /kaggle/working/outputs
folders = os.listdir()
for folder in folders:
    print(folder)
    files = os.listdir(folder)
    for path in files:
        name= os.path.basename(path)
        path = os.path.join('outputs',folder,path)
        html=f'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href={path}>{name}</a>'
        display(HTML(html))


# Create the model with NeuS
The output model is in /kaggle/working/Wonder3D/NeuS/exp/neus/out/meshes

In [ ]:
%cd /kaggle/working/Wonder3D/NeuS
!bash run.sh /kaggle/working/Wonder3D/outputs/ out

---
# FileBrowser

### Install and configure FileBrowser

In [ ]:
%cd /kaggle
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/filebrowser
!rm /kaggle/config.json
!/kaggle/filebrowser config init 
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/filebrowser users add admin admin > /dev/null
!/kaggle/filebrowser config export "/kaggle/working/config.json"
#!cat /kaggle/config.json # show the config file

### Start FileBrowser

* Follow the .remote.moe link to interact with FileBrowser.  
* Outputs are in the folder working -> outputs  
* If the app starts with a login screen, the username and password are both "admin", which was set in the configuration code block above.

In [ ]:
%cd /kaggle

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser -c "/kaggle/config.json" & ssh -R 80:127.0.0.1:8080 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 